In [1]:
import numpy as np
import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import statsmodels.stats.api as sm

C:\Users\evasiliev\AppData\Local\Continuum\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


<font color='green'>
### 1

Версия множителей Лагранжа 

<font color='green'>
### 2

количестве несовпадающих пар значений признаков — (0,1) и (1,0), и объёме выборок n

<font color='green'>
### 3

In [2]:
n_full=50
n_exp=34
n_ctrl=16
m_exp=10
m_ctrl=4

sample_exp = np.append(np.ones(m_exp), (np.zeros(n_exp-m_exp)))
sample_ctrl = np.append(np.ones(m_ctrl), (np.zeros(n_ctrl-m_ctrl)))


def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)
    
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(sample_exp, sample_ctrl), 'greater')

p-value: 0.372930


<font color='green'>
### 4

In [25]:
df_banknotes = pd.read_csv('banknotes.txt', sep='\t')

In [26]:
df_banknotes.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [5]:
#df_banknotes[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']] = ( df_banknotes[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']] - \
#df_banknotes[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']].mean() ) / df_banknotes[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']].std()

In [27]:
X = df_banknotes[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']].as_matrix()
y = df_banknotes[['real']].as_matrix()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=50, random_state=1)
print X_train.shape, X_test.shape, y_train.shape, y_test.shape

(150L, 6L) (50L, 6L) (150L, 1L) (50L, 1L)


In [29]:
LogR1 = LogisticRegression()
LogR1.fit(X_train[:,:3], y_train)
y_pred1 = LogR1.predict(X_test[:,:3]).reshape((X_test.shape[0], 1))
score1 = accuracy_score(y_pred1, y_test)
print score1

0.8


In [30]:
LogR2 = LogisticRegression()
LogR2.fit(X_train[:,3:], y_train)
y_pred2 = LogR2.predict(X_test[:,3:]).reshape((X_test.shape[0], 1))
score2 = accuracy_score(y_pred2, y_test)
print score2

0.98


In [31]:
sample1, sample2, = [], []
for i in xrange(len(y_pred1)):
    sample1.append(y_pred1[i] != y_test[i])
    sample2.append(y_pred2[i] != y_test[i])

def solve_4(sample1, sample2):
    n = 50
    sample = zip(sample1, sample2)
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    z_stat = abs(float(f - g) / np.sqrt(f + g - float((f - g)**2) / n ))
    
    return 1 - scipy.stats.norm.cdf(z_stat)

print solve_4(sample1, sample2)

0.00164846922778


<font color='green'>
### 5

In [32]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

print "95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(sample1, sample2)

95% confidence interval for a difference between proportions: [0.059945, 0.300055]


<font color='green'>
### 6

In [19]:
mean_gen = 525
std_gen = 100
mean_courses = 541.4
n_courses = 100
z_stat = (mean_courses - mean_gen) / (std_gen / np.sqrt(n_courses))
print 1 - scipy.stats.norm.cdf(z_stat)

0.0505025834741


<font color='green'>
### 7

In [20]:
mean_gen = 525
std_gen = 100
mean_courses = 541.5
n_courses = 100
z_stat = (mean_courses - mean_gen) / (std_gen / np.sqrt(n_courses))
print 1 - scipy.stats.norm.cdf(z_stat)

0.0494714680336
